In [37]:
from functools import partial

import astroplan as ap
import astropy.units as u
import numpy as np

from astropaul.database import database_connection, html_path
import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
session = tlc.ObservingSession(ap.Observer.at_site("LBT"))
# session.add_day_range("2025-09-29", "2025-10-02")
# session.add_day_range("2025-10-18", "2025-10-22")
# session.add_day_range("2025-11-14", "2025-11-20")
# session.add_day_range("2025-12-19", "2025-12-23")
# session.add_day_range("2026-01-14", "2026-01-18")
session.add_day_range("2026-01-27", "2026-01-31")

science_target_types = {"QuadEB", "SextEB"}
calibration_target_types = {"RV Standard", "Telluric Standard"}

phase_event_defs = [
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.0)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.05)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.18)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.32)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.45)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.55)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.68)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.82)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.95)),
]

pepsi_args = {
    "fiber": "300",
    "cd_blue": 3,
    "cd_red": 6,
    "snr": 100,
    "binocular": True,
    "priority": "(see grid)",
}

name = f"LBT Observing {session.time_range[0].iso[:10]}"

html_dir = html_path() / name
html.clear_directory(html_dir)

science_target_steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: df["Target Type"].isin(science_target_types)),
    tlc.add_lists,
    partial(tlc.filter_targets, criteria=lambda df: df["List LBT 2025B"]),
    tlc.ancillary_data_from_tess,
    partial(tlc.add_database_table, table_name="Ephemerides"),
    tlc.tabulate_ephemerides,
    partial(tlc.add_database_table, table_name="PEPSI Observations"),
    partial(tlc.filter_targets, criteria=lambda df: df["Num PEPSI Observations"] > 0),
    partial(tlc.add_observability, observing_session=session),
    partial(lbt.add_pepsi_params, **dict({"num_exp": 2}, **pepsi_args)),
]

calibration_target_steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: df["Target Type"].isin(calibration_target_types)),
    tlc.ancillary_data_from_tess,
    partial(tlc.add_columns_from_sql, table_name="RV Calibration Targets", primary_cols=["RV"]),
    partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["Teff"].isna()),
    partial(tlc.add_observability, observing_session=session, time_resolution=2 * u.hour),
    partial(lbt.add_pepsi_params, **dict({"num_exp": 1}, **pepsi_args)),
]

common_steps = [
    partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["Teff"].isna()),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
    tlc.add_tess_catalog_associations,
    partial(tlc.filter_targets, criteria=lambda df: df["PEPSI exp_time"] < 600),
]

with database_connection() as conn:
    creator = tlc.TargetListCreator(connection=conn, phase_event_defs=phase_event_defs)
    science_targets = creator.calculate(name="Science Targets", steps=science_target_steps + common_steps)
    calibration_targets = creator.calculate(name="Calibration Targets", steps=calibration_target_steps + common_steps, verbose=False)
    tl = tlc.TargetList.union([science_targets, calibration_targets], name=name)
    tl = creator.calculate(initial_list=tl, steps=[partial(lbt.assign_rv_standards, target_types=science_target_types)])
tl.target_list["PEPSI notes"] = tl.target_list["Target Type"]
readme = lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session)

print(tl.summarize())
# tl.target_list

Name: LBT Observing 2026-01-27
Criteria:
  Union of lists: Science Targets and Calibration Targets:
    List Science Targets:
      All targets loaded (732 targets remain)
      lambda df: df["Target Type"].isin({'SextEB', 'QuadEB'}) (322 targets remain)
      lambda df: df["List LBT 2025B"] (32 targets remain)
      lambda df: df["Num PEPSI Observations"] > 0 (16 targets remain)
      Observability calculated at LBT in 15.0 min intervals from 2026-01-27 to 2026-01-31
        AltitudeConstraint: {'min': 30.0, 'max': 80.0, 'boolean_constraint': True}
      Inverse of: lambda df: df["Teff"].isna() (16 targets remain)
      lambda df: (df["Observable Any Night"]) (16 targets remain)
      lambda df: df["PEPSI exp_time"] < 600 (16 targets remain)
    List Calibration Targets:
      All targets loaded (732 targets remain)
      lambda df: df["Target Type"].isin({'RV Standard', 'Telluric Standard'}) (394 targets remain)
      Inverse of: lambda df: df["Teff"].isna() (392 targets remain)
    

In [39]:
illumination_categories = [
    ((0.0, 0.4), "Dark"),
    ((0.4, 0.7), "Gray"),
    ((0.7, 1.0), "Bright"),
]

distance_categories = {
    "Dark": [
        ((0, 180), 1),
    ],
    "Gray": [
        ((0, 5), 0.1),
        ((5, 15), 0.85),
        ((15, 180), 1),
    ],
    "Bright": [
        ((0, 15), 0.25),
        ((15, 30), 0.75),
        ((30, 180), 1),
    ],
}

altitude_categories = [
    ((-90, 35), 0),
    ((35, 45), 0.95),
    ((45, 90), 1),
]

phase_scores = {
    "B|B": 0.5,
    "B|R": 0.8,
    "B|T": 0.4,
    "R|B": 0.8,
    "R|R": 0.9,
    "R|T": 1,
    "T|B": 0.4,
    "T|R": 1,
    "T|T": 0.1,
}

ephem_categories = [
    ((3.5, 99), 1),
    ((2.1, 3.5), .9),
    ((0, 2.1), .7),
]

science_tl = tl.copy()
science_tl.target_list = science_tl.target_list[science_tl.target_list["Target Type"] == "QuadEB"]
pl = pr.PriorityList(science_tl, session, interval=30 * u.min)
pr.calculate_moon_priority(pl, illumination_categories=illumination_categories, dist_categories=distance_categories)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.calculate_ephemerides_priority(pl, categories=ephem_categories)
pr.calculate_list_priority(pl, "HQND", false_value=0.75)
pr.calculate_phase_priority(pl, phase_defs=phase_event_defs, phase_categories=phase_scores)
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.3)
pl.categorize_priorities(
    bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"]
)

 '2026-01-27T02:00:00.000000000' '2026-01-27T02:30:00.000000000'
 '2026-01-27T03:00:00.000000000' '2026-01-27T03:30:00.000000000'
 '2026-01-27T04:00:00.000000000' '2026-01-27T04:30:00.000000000'
 '2026-01-27T05:00:00.000000000' '2026-01-27T05:30:00.000000000'
 '2026-01-27T06:00:00.000000000' '2026-01-27T06:30:00.000000000'
 '2026-01-27T07:00:00.000000000' '2026-01-27T07:30:00.000000000'
 '2026-01-27T08:00:00.000000000' '2026-01-27T08:30:00.000000000'
 '2026-01-27T09:00:00.000000000' '2026-01-27T09:30:00.000000000'
 '2026-01-27T10:00:00.000000000' '2026-01-27T10:30:00.000000000'
 '2026-01-27T11:00:00.000000000' '2026-01-27T11:30:00.000000000'
 '2026-01-27T12:00:00.000000000' '2026-01-27T12:30:00.000000000'
 '2026-01-27T13:00:00.000000000' '2026-01-27T13:30:00.000000000'], obsgeoloc=[( 4609912.34672212,  2778262.79895543, 3415914.89207045),
 ( 4206726.00124672,  3356577.24938063, 3416922.02043953),
 ( 3731317.44417653,  3877147.24980334, 3418114.96420406),
 ( 3191865.58589921,  4331016.9

In [40]:
readme = lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session)
with open("UVa_Multistar.README", "w") as f:
    f.write(readme)

other_files = {
    "LBT Readme": lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session),
    "LBT CSV": open("UVa_Multistar.csv", "r").readlines(),
}

html.render_observing_pages(tl, pl, other_files, html_dir)
# pl.categorical_priorities[0]

In [6]:
# import asyncio
# import glob
# # for categorical_file in glob.glob("*.*"):

# for categorical_file in glob.glob("../../Observing Files/LBT Observing 2025-07-04/Categorical Priorities *.html"):

#     await html.html_to_pdf(categorical_file, categorical_file.replace(".html", ".pdf"))
#     print(categorical_file)